In [5]:
# =========================
# DailyArtAgent (single-input tools) — full solution
# deps: requests, pydantic, langchain, langchain-openai, langchain-community, python-dotenv
# =========================
import os
import json
import random
import requests
import datetime as dt
from pathlib import Path
from typing import Any, Dict, List, Optional, Type

from dotenv import load_dotenv
from pydantic import BaseModel, Field

from langchain_openai import ChatOpenAI
from langchain.tools import BaseTool, tool
from langchain.agents import Tool, AgentType, AgentExecutor, initialize_agent
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.prompts import ChatPromptTemplate


# ---------- ENV ----------
load_dotenv(dotenv_path=Path(".env"))
assert os.getenv("OPENAI_API_KEY"), "Missing OPENAI_API_KEY in .env"


# ---------- SINGLE-INPUT TOOLS ----------
class HTTPGetJSONTool(BaseTool):
    """
    GET a JSON endpoint.
    Input is a SINGLE STRING containing JSON: {"url":"...", "params": {...}}
    This avoids the 'ZeroShotAgent does not support multi-input tool' error.
    """
    name: str = "http_get_json"
    description: str = "GET JSON. Input: JSON string with keys 'url' and optional 'params'."

    def _run(self, query: str) -> str:
        try:
            data = json.loads(query)
            url = data["url"]
            params = data.get("params", {})
            r = requests.get(url, params=params, timeout=15)
            r.raise_for_status()
            return r.text
        except Exception as e:
            return json.dumps({"status": "error", "error": str(e), "query": query})

    def _arun(self, **kwargs):
        raise NotImplementedError


class MetObjectTool(BaseTool):
    """
    The Met Museum: fetch object by objectID.
    Input is a STRING object_id, e.g. "437853"
    """
    name: str = "met_get_object"
    description: str = "Fetch Met object JSON by object_id (string). Input example: '437853'"

    def _run(self, object_id: str) -> str:
        try:
            oid = int(object_id.strip())
            url = f"https://collectionapi.metmuseum.org/public/collection/v1/objects/{oid}"
            r = requests.get(url, timeout=15)
            r.raise_for_status()
            return r.text
        except Exception as e:
            return json.dumps({"status": "error", "error": str(e), "object_id": object_id})

    def _arun(self, **kwargs):
        raise NotImplementedError


class CMASearchTool(BaseTool):
    """
    Cleveland Museum of Art search: q -> first result JSON.
    Input is a STRING query, e.g. "Monet Water Lilies"
    """
    name: str = "cma_search"
    description: str = "Search CMA (first result). Input: plain string query."

    def _run(self, q: str) -> str:
        try:
            url = "https://openaccess-api.clevelandart.org/api/artworks"
            r = requests.get(url, params={"q": q, "limit": 1}, timeout=15)
            r.raise_for_status()
            return r.text
        except Exception as e:
            return json.dumps({"status": "error", "error": str(e), "q": q})

    def _arun(self, **kwargs):
        raise NotImplementedError


@tool
def serper_search(query: str) -> str:
    """
    Optional: Google results via Serper.dev.
    Input: plain query string.
    Requires SERPER_API_KEY in .env.
    """
    key = os.getenv("SERPER_API_KEY")
    if not key:
        return json.dumps({"status": "error", "error": "SERPER_API_KEY not set"})
    url = "https://google.serper.dev/search"
    headers = {"X-API-KEY": key, "Content-Type": "application/json"}
    try:
        resp = requests.post(
            url, headers=headers,
            data=json.dumps({"q": query, "gl": "us", "hl": "en"}), timeout=15
        )
        resp.raise_for_status()
        return resp.text
    except Exception as e:
        return json.dumps({"status": "error", "error": str(e), "query": query})


# ---------- GENERATOR ----------
class DailyPostGenerator:
    """
    Generates a concise 3–5 sentence post (120–180 words) with inline [1][2] citations
    from museum metadata + Wikipedia summary. Returns JSON.
    """

    def __init__(self, model: str = "gpt-4o-mini"):
        self.llm = ChatOpenAI(model=model, temperature=0.4)
        self.template = ChatPromptTemplate.from_messages([
            ("system",
             "You are an art writer for a 'Daily Art Piece'. Produce 120–180 words in 3–5 sentences. "
             "Be vivid and precise, avoid clichés. Include inline citations like [1], [2] that map to provided sources. "
             "No bullet lists in body. Return JSON with fields: "
             "title, post_markdown, genre, image_url, artwork_id, artist, artist_info, year, museum, artist_quote, citations[]."),
            ("human",
             "ARTWORK METADATA JSON:\n{art_json}\n\n"
             "WIKIPEDIA SUMMARY (optional):\n{wiki_summary}\n\n"
             "Required structure (3–5 sentences):\n"
             "1) Hook (mood/why it matters)\n"
             "2) Essentials (title, artist, date, medium) + 1 notable detail\n"
             "3) History insight (provenance/movement/exhibition)\n"
             "4) Visual analysis (composition/color/technique)\n"
             "5) Optional today-connection\n\n"
             "Citations:\n"
             "[1] {src1_label} — {src1_url}\n"
             "[2] {src2_label} — {src2_url}\n")
        ])

    def generate(self, art_json: Dict[str, Any], wiki_summary: str,
                 sources: List[Dict[str, str]]) -> Dict[str, Any]:
        # Normalize default sources mapping: [1] museum, [2] wikipedia
        src1 = sources[0] if len(sources) > 0 else {"label": "Museum", "url": ""}
        src2 = sources[1] if len(sources) > 1 else {"label": "Wikipedia", "url": ""}

        chain = self.template | self.llm
        resp = chain.invoke({
            "art_json": json.dumps(art_json, ensure_ascii=False),
            "wiki_summary": wiki_summary or "",
            "src1_label": src1.get("label", "Museum"),
            "src1_url": src1.get("url", ""),
            "src2_label": src2.get("label", "Wikipedia"),
            "src2_url": src2.get("url", ""),
        }).content

        # Parse LLM JSON robustly
        try:
            data = json.loads(resp)
        except json.JSONDecodeError:
            import re
            m = re.search(r"\{.*\}", resp, re.DOTALL)
            data = json.loads(m.group()) if m else {"title": "Untitled", "post_markdown": resp, "citations": []}

        # Backfill common fields if missing
        data.setdefault("image_url", art_json.get("primaryImageSmall") or art_json.get("primaryImage") or "")
        data.setdefault("artwork_id", str(art_json.get("objectID") or art_json.get("id") or ""))
        data.setdefault("artist", art_json.get("artistDisplayName") or art_json.get("creators", ""))
        data.setdefault("work_title", art_json.get("title", ""))
        data.setdefault("year", art_json.get("objectDate") or art_json.get("dated", ""))
        data.setdefault("museum", "The Metropolitan Museum of Art" if art_json.get("objectID") else "Cleveland Museum of Art")

        if not data.get("citations"):
            cites = []
            if src1.get("url"): cites.append({"n": 1, "label": src1["label"], "url": src1["url"]})
            if src2.get("url"): cites.append({"n": 2, "label": src2["label"], "url": src2["url"]})
            data["citations"] = cites
        return data


# ---------- AGENT ----------
class DailyArtAgent:
    """
    - Initializes single-input tools (Met, CMA, generic HTTP, Wikipedia)
    - Picks an artwork (Met by default; you can swap strategy)
    - Fetches metadata + wiki context
    - Generates a daily post with citations via DailyPostGenerator
    """

    def __init__(self, model: str = "gpt-4o-mini"):
        self.llm = ChatOpenAI(model=model, temperature=0.2)
        self.tools = self._initialize_tools()
        self.agent: AgentExecutor = initialize_agent(
            tools=self.tools,
            llm=self.llm,
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # works with single-input tools
            verbose=False,
            handle_parsing_errors=True,
        )
        self.generator = DailyPostGenerator(model=model)

        # Replace with your curated/random list
        self.met_ids = [436121, 459055, 436839, 437853, 436532]

    def _initialize_tools(self) -> List[Tool]:
        wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1500))
        return [
            MetObjectTool(),
            CMASearchTool(),
            HTTPGetJSONTool(),
            Tool(
                name="wikipedia",
                func=wiki.run,
                description="Fetch Wikipedia summary for an artist/work (input: plain query string)."
            ),
            # serper_search,  # enable if you want extra web context (needs SERPER_API_KEY)
        ]

    # ----- Selection / Fetch -----
    def pick_met_object_id(self) -> int:
        return random.choice(self.met_ids)

    def fetch_met_object(self, object_id: int) -> Dict[str, Any]:
        raw = MetObjectTool()._run(str(object_id))
        try:
            return json.loads(raw)
        except Exception:
            return {}

    def compile_sources(self, art: Dict[str, Any]) -> List[Dict[str, str]]:
        sources = []
        if art.get("objectURL"):
            sources.append({"label": "The Met", "url": art["objectURL"]})
        # [2] is Wikipedia; added in generator labels
        return sources

    # ----- Orchestration -----
    def run_daily(self) -> Dict[str, Any]:
        oid = self.pick_met_object_id()
        art = self.fetch_met_object(oid)
        if not art or not art.get("title"):
            raise RuntimeError("Failed to fetch Met object metadata")

        wiki_q = f"{art.get('title','')} {art.get('artistDisplayName','')}".strip()
        wiki_summary = WikipediaQueryRun(
            api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1500)
        ).run(wiki_q)

        sources = self.compile_sources(art)
        post = self.generator.generate(art, wiki_summary, sources)
        return post



In [6]:


agent = DailyArtAgent(model="gpt-4o-mini")
post = agent.run_daily()

print(post)

print("===", post.get("title", "Untitled"), "===")
print(post.get("post_markdown", ""))

citations = post.get("citations", [])
print("\nCitations:")
for idx, c in enumerate(citations, start=1):
    if isinstance(c, dict):
        print(f"[{c.get('n', idx)}] {c.get('label','')} : {c.get('url','')}")
    else:
        # c is probably a string
        print(f"[{idx}] {c}")

{'title': 'The Annunciation by Hans Memling', 'post_markdown': "In a moment suspended between the divine and the earthly, Hans Memling's *The Annunciation* (1480–89) captures the profound intersection of faith and emotion. This oil on panel painting, now transferred to canvas, depicts the moment the Archangel Gabriel announces to the Virgin Mary her miraculous conception. Memling's work, celebrated for its startling originality, emphasizes Mary's purity and foreshadows her future as the mother of Jesus, a theme that resonates deeply within Christian iconography [1][2]. The composition is rich with detail: Gabriel, adorned in ecclesiastical robes, stands amidst a serene domestic interior, while a dove, symbolizing the Holy Spirit, hovers above Mary. This painting not only reflects the artist's mastery of color and light but also serves as a poignant reminder of the sacred moments that shape human history.", 'citations': ['https://www.metmuseum.org/art/collection/search/459055', 'https:/